# Assessment 2 
During the covid pandemic, governments tried to predict the levels of infection so that they could manage the required infrastructure, especially hospital beds and (sadly) mortuary places. In this mini-project, you will take data from two different, publicly available, sources and try to see how well you could have done if you had been working for the UK government.  One of the sources is the UK data from the period and the other is google’s data on our behaviour during the pandemic.

You will use data up to a specific date to predict hospital beds needed, and deaths that will occur  in one, two, three and four weeks from that date.  You will do this for the whole pandemic period i.e. using a rolling  window. You will also quantify how well your prediction works.

Things that you might want to consider in your analysis include:

- There will be a time lag between people’s  behaviour changing and the number of people requiring a hospital bed (and another between people requiring hospitalisation and them dying).

- Both sources of data will have information that is  of no use to you and you will have to investigate those sources that are useful to you. You should start by thinking this through and then trying the effect of different variable.

- Both data sets may require some cleaning and (perhaps) some smoothing to remove effects such as weekends etc.

- As the situation changed some aspects of the older data nay or may not continue to be useful.  

- You may wish to do this by region or for the whole country, or even to compare the two.

We will provide you with basic code to access the data. It is up to you if you use it or not.


## 1. Setup

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

import matplotlib.pylab as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
#Show plots inline
%matplotlib inline

#Some date formatters to use for plotting 
weeks = mdates.WeekdayLocator()
years = mdates.YearLocator()
months = mdates.MonthLocator()
weekdays = mdates.DayLocator()
dateFmt = mdates.DateFormatter('%b-%y')

In [2]:
#seaborn is a useful package for handling graphics and 
#producing publication quality images with better colour schemes
import seaborn as sns
sns.set()
sns.set_style("white")
sns.set_context("talk")
#sns.set(font_scale=1.5) 
#sns.set_context("poster")

plt.rcParams.update({'font.size': 32})
plt.rcParams.update({'lines.linewidth':3})

## 2. Predictors

In principle, covid cases, hospital admissions, and deaths are time-lagged outcomes of covariates or "predictor variables" such as the level of interactions in the population, mobility metrics, levels of restrictions etc.

A data set which is often used as a possible predictor is google mobility metrics https://www.google.com/covid19/mobility/.

In [3]:
#Import the data from Google.
#This is a large file!
df_google = pd.read_csv('/home/sm219/Downloads/Global_Mobility_Report.csv', low_memory = False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/sm219/Downloads/Global_Mobility_Report.csv'

In [ ]:
#Select `United Kingdom` from the full data.
df_google = df_google[df_google['country_region']=='United Kingdom']
#Discard sub regions
#Note - we England vs UK
df_google = df_google[df_google['sub_region_1'].isnull()]

In [ ]:
#Re-index by data
df_google = df_google.set_index(pd.to_datetime(df_google['date']))
df_google = df_google.sort_index()
df_google.head()


Plot up some of the metrics. 

In [ ]:
fig, ax = plt.subplots(figsize=(15,10),sharex= True, facecolor='white')

ax.plot(df_google.index,df_google['workplaces_percent_change_from_baseline'],label='Work')
ax.plot(df_google.index,df_google['transit_stations_percent_change_from_baseline'],label='Public Transport')
ax.plot(df_google.index,df_google['parks_percent_change_from_baseline'],label='Parks')


ax.tick_params(axis="both", direction="in", which="both", right=True,left=True, top=True, bottom=True)


ax.grid(which='both')
ax.set_xlabel('Date')
ax.set_ylabel('Fraction relative to Pre-Covid')

# format the ticks
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(dateFmt)
ax.xaxis.set_minor_locator(weeks)
#ax.set_xlim(left=dt.datetime(2021,7,1))
ax.set_title('UK Workplace Google Metric')
ax.legend()

_= fig.autofmt_xdate()

plt.tight_layout()
plt.savefig('mobility.png')
plt.show()

## The UK data from the covid period.

The UK data is available as a zip file from:

https://ukhsa-dashboard.data.gov.uk/covid-19-archive-data-download

this then needs to be unzipped 
```console
unzip  covid-19-archive.zip
```
This should produce a series of directories:
```console
drwxr-xr-x. 1 collngdj collngdj        126 Oct 25 16:08 Cases
-rw-r--r--. 1 collngdj collngdj 1592889236 Oct 25 16:07 covid-19-archive.zip
drwxr-xr-x. 1 collngdj collngdj        126 Oct 25 16:08 Deaths
drwxr-xr-x. 1 collngdj collngdj        126 Oct 25 16:08 Healthcare
drwxr-xr-x. 1 collngdj collngdj        126 Oct 25 16:08 Testing
drwxr-xr-x. 1 collngdj collngdj        126 Oct 25 16:08 Vaccinations
```

each of these has a set of csv files arranged over different years.

The following code is an example of how you might want to read these in and concatinate them over different years.



In [ ]:
import re
from enum import StrEnum, auto
import pathlib
import pandas as pd

class DataType(StrEnum):
    Vaccinations = auto()
    Cases = auto()
    Deaths = auto()
    Healthcare = auto()
    Testing = auto()
    


def csvconcat(datatype: DataType,
              metric: str | None = None,
              dataroot: str | pathlib.Path = pathlib.Path.cwd()) -> dict[str, pd.DataFrame]:
    """
    Get Pandas DataFrames from the UK COVID-19 .csv data, Concatenated across years.

    Args:
        datatype (DataType): Corresponds to the directory to scan use enum type above.
                             i.e. [DataType.Cases|DataType.Deaths|DataType.Healthcare|
                                   DataType.Testing|DataType.Vaccinations]
        metric (str | None, optional): Chose an individual metric to process. If None (the default) then all
                                       metrics for the given DataType are processed. Defaults to None.
        dataroot (pathlib.Path, optional): The root directory for the unpacked UK COVID-19 data.
                                           DataType directories (Cases/Deaths) etc should be under this root.
                                           Defaults to pathlib.Path.cwd().
    Raises:
        ValueError: If there is a failure to convert the given dataroot to a Path object.

    Returns:
        dict[str, pd.DataFrame]: The mapping from metric to fully concatenated DataFrame.
    """
    if not isinstance(dataroot, pathlib.Path):
        try:
            dataroot = pathlib.Path(dataroot)
        except:
            raise ValueError(f"dataroot: '{dataroot}' could not be converted to a Path.")

    dataroot /= datatype.name
    metric_regex = re.compile(r"(?P<metric>\w+?)_(?P<specifier>nation|region|utla|ltla|overview)_20\d\d.csv")
    metrics: set[str] = {str(metric)}
    if metric is None:
        file_list = (file_.relative_to(dataroot).name for file_ in dataroot.rglob("*.csv"))
        metrics = {match.group("metric") for file_ in file_list if (match := metric_regex.match(file_))}

    ret = {}
    for current_metric in metrics:
        ret.update({current_metric: pd.concat(pd.read_csv(file_) for file_ in dataroot.rglob(f"{current_metric}_*.csv"))})

    return ret

In [ ]:
# Get a specific metric
path = "/home/sm219/Downloads/covid-19-archive"
data = csvconcat(DataType.Healthcare, "hospitalCases", dataroot = path)
print(list(data.keys()))
type(data['hospitalCases'])

In [ ]:
# Get all metrics for a DataType
data = csvconcat(DataType.Healthcare, dataroot = path)
print(f"Data contains {len(data)} concatenated metrics")
#print(f"Checking our chosen metric 'transmissionRateGrowthRateMax' is found in data: {'transmissionRateGrowthRateMax' in data}")
print("Available metrics:")
print(list(data.keys()))

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.custombg {
    background-color: green;
    color: white;
}
</style>
""")

# Start of project

<div style="background-color:#C2F5DD">

# Introduction

Initially, the goal is to do some exploratory data analysis to get a feel of the different datasets and the various features and data types present in them. This is done using `df.head()` and `df.info()` to look at the data stored inside them. The prediction for the COVID beds and COVID deaths were done seperately because either of them could have been a predictor for the other one.

**MARKER: Adjust `path` variable accordingly to load data.**

<div style="background-color:#C2F5DD">
Firstly, some of the datsets were examined and the data inside was plotted to get a feel for the dataset. Different metrics were plotted just by changing the index.

In [ ]:
metric = list(data.keys())[10]
df_covid = data[metric]

df_covid.info()

<div style="background-color:#C2F5DD">

Since we are doing time-series analysis it is better to set the datetime column as the index so it is easier to select data via dates.

In [ ]:
#Re-index by data
df_covid.set_index(pd.to_datetime(df_covid['date']),inplace=True)
df_covid.sort_index(inplace=True)
df_covid.columns

df_covid.head()

In [ ]:
plt.figure(figsize=(10, 5))  
plt.plot(df_covid.index, df_covid["value"], label = f"{metric} data")
plt.grid(True)
plt.xlabel("Data")
plt.ylabel("Value")
plt.legend(loc="best")
plt.yscale("log")
plt.xticks(rotation=90)
plt.show()

<div style="background-color:#C2F5DD">

## Feature Engineering

To allow our ML model to have the most predictive power, one of the most important steps is to feed the ML model the appropriate features so it can learn the complex relationships between the features and the target variable. Due to the high dimensionality and associated complexity of the data, the best approach will be to use Random Forest Regressor or XG Boost Regressor. These models have the capability to handle complex data whilst maintaining a managable computational cost. However, the list of all features from all the healthcare, testing, deaths, vaccinations, cases database is large. Therefore, to reduce the features down to a manageble level, some feature engineering has to be performed. Either LASSO or PCA can be used to reduce the dimensionality of the data. However, some features can already be removed based off requirements.

### Initial Removed features

**cum**: Any variable that are cumulative are irrelevant for the prediction of covid beds/deaths. The cumulative cases, or the cumulative admissions would not affect the current rolling prediction of covid beds/deaths. *Note: Exception applies to vaccination and potentially testing data where the cumulative vaccinations (1st dose/ 2nd dose etc.) would indicate a lower probability of (re)infection for those people. So all the cumulative data was preserved only in the vaccination and testing dataset.*

**weekly**: All the weekly data was removed because the data was aggregated for the week thus making it irrelevant when there is daily data already available.

**archive**: Any archived data was removed because those datapoints were from the past so not relevant when making current predictions.

**publishdate**: In the testing datasets especially, many features had an equivalent publish date which was the date when the case/infection/death got recorded. Again, when the case/infection/death was regarded is irrelevant. Only the date of, say, death matters for predictions.

**registrationdate**: Similar to publish date, registration date is another formality of when the data was recorded and again not relevant for us.

**direction**: The direction features in the datasets indicate whether a certain feature is going up or down, which is not really important considering we already have numerically data. Furthermore, this will be highly correlated with the numerical data and provide no additional information but rather add unecessary features which will increase the computational complexity.

**change/percentage**: This provides the change or percentage change which is ignored for now as we have the numerical data available already. This is a sort of 50/50 call to reduce the number of features. Latter we can try including these featurs to see the change that the features bring to the predicitve power of the model, if any.

For vaccination and testing - two new considerations were included:

**cum**: For vaccination and testing it does matter how many new people have been tested/vaccinated rather the total sum is more important. So cumulative features were left in.

**new**: The daily change/the new number of people infected does not really matter. Only the total sum matters becaues it represents the people that are less likely to get infected. So any new change features were left out.

For vaccinations, one final additon:

**dose**: For vaccinations we are mainly interested in the total dosage given to people. The vaccinations and testing datasets are similar and probably closely correlated so we are only interested in a few features from the vaccination dataset such as the dosage.




<div style="background-color:#C2F5DD">

A function called `transform_date` to group the data via date as we are considering national cases of covid rather than regional based data. We are mainly interested in the date and the value thus we can discard other columns such as the region and the metric name.

Another function called `check_na` was used to make sure that the features used do not have *NaN* values for the date range that we are interested in.

On another note, the processing for the data was done in batches to prevent loading up large files at the same time which caused memory issues resulting in jupyter notebooks crashing.

In [ ]:
def transform_data(data, metric_name, start = '2020-06-01', end = '2022-09-30', get_length = False):
    """
    Retrieve the dataframe from the NHS dataset, aggregate the data by date, use datetime as the index,
    and remove any archive data.

    Parameters:
    metric_name (str): A string to describe which metric for which we wish to create a dataframe.
    data (dict): A dictionary containing the data.
    start (Timestamp): A date-time value_cumCasesBySpecimenDate  852 format string to denote the start date of the data.
    end (Timestamp): A date-time format string to denote the end date of the data.
    get_length (Boolean): Returns the length of the metric instead of a dataframe.

    Returns:
    df_temp (Dataframe): A dataframe which is preprocessed such that there are no archives and the data is grouped by date.
    """
    df_temp = data[metric_name]
    
    
    if 'metric_name' in df_temp.columns:
        df_temp['metric_name'] = df_temp['metric_name'].astype(str)  # Ensure it's string type
        df_temp = df_temp[~df_temp['metric_name'].str.contains("archive", case=False, na=False)]

    
    if 'date' in df_temp.columns:
        df_temp['date'] = pd.to_datetime(df_temp['date'])
        df_temp.set_index('date', inplace=True)
    
    
    df_temp = df_temp.groupby('date')['value'].sum().reset_index() #collect all data
    df_temp.set_index('date', inplace=True)

    start_date = pd.Timestamp(start)
    cutoff_date = pd.Timestamp(end)
    #Only return the date range we are examining
    df_temp = df_temp.loc[(df_temp.index >= start_date) & (df_temp.index <= cutoff_date)]

    df_temp = df_temp.rename(columns={"value": f"value_{metric_name}"})  # Rename value column

    if get_length:
        return len(df_temp[f"value_{metric_name}"])
    
    return df_temp


In [ ]:
def check_na(data, metrics_list, target_length, start = '2020-06-01', end = '2022-09-30'):
    '''
    Loop through the dataframe the make sure the size of the data are as required.

    Parameters:
    data (dict): A dictionary containing the data.
    metrics_list (list): A list of all the metric names in data.
    target_length (int): An integer describing the required length of the dataframe.
    start (Timestamp): A date-time format string to denote the start date of the data.
    end (Timestamp): A date-time format string to denote the end date of the data.

    Returns:
    na_list (list): A list of columns to remove because they contain NaN values.
    '''
    na_list = []
    start_date = pd.Timestamp(start)
    cutoff_date = pd.Timestamp(end)

    for metric in metrics_list:
        df = data[metric]

        if 'metric_name' in df.columns:
            df['metric_name'] = df['metric_name'].astype(str) 
            df = df[~df['metric_name'].str.contains("archive", case=False, na=False)]

    
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
            df = df.set_index('date')
        elif not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)

        df = df.groupby('date')['value'].sum().reset_index()
        df = df.set_index('date')

        df = df.loc[(df.index >= start_date) & (df.index <= cutoff_date)]

        if "value" in df.columns:
            total_entries = len(df["value"])
            

            if (total_entries != target_length): #if there are too few datapoints remove those columns
                na_list.append(metric)

    return na_list


In [ ]:
data_healthcare = csvconcat(DataType.Healthcare, dataroot = path) #Loading the data first
data_cases = csvconcat(DataType.Cases, dataroot = path)
data_deaths = csvconcat(DataType.Deaths, dataroot = path)

In [ ]:
target_beds = transform_data(data = data_healthcare, metric_name = "hospitalCases", get_length = True) #get target beds length
print(target_beds) #can use same value for beds and deaths

In [ ]:
na_col_list = check_na(data_healthcare, metrics_list = list(data_healthcare.keys()), target_length = target_beds)
print(na_col_list)
print(len(na_col_list))
print(len(list(data_healthcare.keys()))) #checking if check_na works as intended

In [ ]:
df = transform_data(data_cases, list(data_cases.keys())[4]) #checking if transform_data works as inteded.
df.head()

In [ ]:
data_list = [data_healthcare, data_cases, data_deaths] #combining in batches

<div style="background-color:#C2F5DD">

Combining all the dataframes together and discarding some obvious ones which are not going to be useful for our ML project as discussed above. To reduce the number of features, LASSO can be used to identify the most important features in the ML model. So the features that were ambigious as to whether they would help or not have been included. More advanced feature selection is done later.

In [ ]:
from tqdm import tqdm

def process_datasets(data_list, unwanted_terms, target_column, wanted_terms = None):
    """
    Processes multiple datasets to filter, transform, and concatenate them based on specified conditions.

    Parameters:
    - data_list (list): A list of datasets (Dictionaries or DataFrames) to process.
    - check_na (function): Function to check for non-NA percentage in datasets.
    - transform_data (function): Function to transform data based on specified metrics.
    - unwanted_terms (list): List of strings with terms to filter out unwanted metrics.
    - target_column (str): Name of the target column for NA checks. 

    Returns:
    - combined_df (DataFrame): A combined DataFrame after processing all metrics and datasets.
    """
    dfs = []  # list of dataframes to concatenate

    for selected_data in tqdm(data_list, desc="Processing DataTypes..."):
        metric_list = list(selected_data.keys())
        na_col_list = check_na(data=selected_data, metrics_list=metric_list, target_length=target_beds)
        
        
        filtered_metrics = [
            metric for metric in metric_list
            if not any(term in metric.lower() for term in unwanted_terms) and metric not in na_col_list and
            (wanted_terms is None or any(term in metric.lower() for term in wanted_terms))
        ]

        
        for metric in tqdm(filtered_metrics, desc='Processing metrics...'):
            transformed_df = transform_data(data=selected_data, metric_name=metric)
            
            if transformed_df is not None:
                dfs.append(transformed_df)

    
    combined_df = pd.concat(dfs, axis=1)
    print(f"Combined DataFrame shape: {combined_df.shape}")
    return combined_df


In [ ]:
combined_df = process_datasets(
    data_list=data_list,  
    unwanted_terms=["cum", "change", "direction", "percentage", "weekly", "archive", "publishdate", 'registrationdate'], 
    target_column='covidOccupiedMVBeds'
)

<div style="background-color:#C2F5DD">

Processing the data in batches to prevent the notebook from crashing by loading the vaccinations and testing data seperately. Also, we included cumulative features for testing and vaccinations because the total number of people with, for example, 1st dose of vaccination would imply they are less likely to get infected so less likely to be admitted to hospital for covid as discussed above. Thus, all the cumulative features were kept for testing and vaccinations.

In [ ]:
del data_cases, data_deaths, data_list, data_healthcare #free up memeory to prevent notebook from crashing

data_testing = csvconcat(DataType.Testing, dataroot = path)
data_list = [data_testing]

test_df = process_datasets(
    data_list=data_list,  
    unwanted_terms=['capacity',"change", "direction", "percentage", "weekly", "archive", "publishdate", 'registrationdate'], 
    target_column='covidOccupiedMVBeds'
)

del data_testing #clear memory

combined_df = pd.concat([combined_df, test_df], axis=1)
combined_df.to_csv('data_without_vacc.csv') #save file for faster loading

In [ ]:
#combined_df = pd.read_csv('data_without_vacc.csv', index_col = 'date')

<div style="background-color:#C2F5DD">

**Marker: Please comment out cell below if any errors**

*Note: Losing the vaccination data would not be too much of a hassle*

In [ ]:
#MARKER: if notebook crashes: comment out this entire cell block

data_vaccinations = csvconcat(DataType.Vaccinations, dataroot = path)

vacc_df = process_datasets(
    data_list=data_list,  
    unwanted_terms=['capacity',"change", "direction", "percentage", "weekly", "archive", "publishdate", 'registrationdate'], 
    target_column='covidOccupiedMVBeds',
    wanted_terms = ['cum', 'dose']
    
)

del data_vaccinations #clear memory

combined_df = pd.concat([combined_df, vacc_df], axis=1)
combined_df.to_csv('data_with_vacc.csv') #save file for faster loading

In [ ]:
combined_df.info()

In [ ]:
combined_df.head()

<div style="background-color:#C2F5DD">

Now the data has been preprocessed, the google dataset can be combined with the original the combined dataframe.

In [ ]:
cutoff_date = pd.Timestamp('2022-09-30') #only predicting upto here
start_date = pd.Timestamp('2020-06-01')    

df_google = df_google[(df_google.index >= start_date) & (df_google.index <= cutoff_date)] #match the dates with the covid data
combined_df = combined_df[(combined_df.index >= start_date) & (combined_df.index <= cutoff_date)]

#remove region labels on goolge dataframe
for column in df_google.columns:
    if "percent" not in column.lower():
        df_google = df_google.drop([column], axis=1)

combined_df = pd.concat([df_google, combined_df], axis=1)

In [ ]:
combined_df.head() #check everything is in order

In [ ]:
# Plotting covid beds in timeframe we are interested

plt.figure(figsize=(10, 5))
plt.plot(combined_df.index, combined_df['value_covidOccupiedMVBeds'], label='COVID Mechanical Ventilation Beds', color='royalblue', linewidth=2)

plt.xlabel('Date')
plt.xticks(rotation = 90)
plt.ylabel('Number of Beds')
plt.title('COVID-19 Mechanical Ventilation Bed Usage Over Time')
plt.grid(True)
plt.show()

In [ ]:
#Same plot for the deaths

plt.figure(figsize=(10, 5))
plt.plot(combined_df.index, combined_df['value_newDailyNsoDeathsByDeathDate'], label='COVID Deaths', color='red', linewidth=2)

plt.xlabel('Date')
plt.xticks(rotation = 90)
plt.ylabel('Number of Deaths')
plt.title('COVID-19 Deaths Over Time')
plt.grid(True)
plt.show()

<div style="background-color:#C2F5DD">
    
## Additional features

To allow the ML models to better predict the seasonal/periodic variations of the data, we can add three new features: day, month, year. The 'day' column will allow the model to understand weekend effects; the 'month' column will allow the model to understand seasonal variations and the year column will aid the model in understanding long time variations. Once, the LASSO is implemented in the next section it will become obvious whether any of these periodic features will be useful or not. These features need to be `OneHotEncoded`. This is done because ML algorithms usually work based on distance. So, for example, the 'days' column assigns 5 for Saturday and 6 for Sunday and 0 for Monday. Thus, the ML model would assume a bigger difference between Sunday and Monday compared to between Saturday and Sunday even though both days are consecutive days to Sunday. `OneHotEncoding` converts this categorical data to numerical data that the ML model can use.

In [ ]:
combined_df.head()

In [ ]:
combined_df["day"] = combined_df.index.dayofweek
combined_df["month"] = combined_df.index.month
combined_df['year'] = combined_df.index.year

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output = False, handle_unknown='ignore')
transform_df = combined_df[['day', 'month', 'year']]
encoded_features = encoder.fit_transform(transform_df) #create encoded df

encoded_df = pd.DataFrame(encoded_features, columns = encoder.get_feature_names_out(['day', 'month', 'year']), 
                          index = combined_df.index) #add them back to original dataframe

combined_df = pd.concat([combined_df, encoded_df], axis=1)
combined_df.drop(['day', 'month', 'year'], axis=1, inplace=True)

combined_df.info()

<div style="background-color:#C2F5DD">

### LASSO feature reduction

Now that the list of features has been shortened, a more sophisticated method can be employed to reduce the number of features further. This is required because if we were to input such high-dimensional data into a ML model, there will be a large risk of overfitting not to mention the large computational cost of performing such a process. Thus, `LASSO` is a linear regression model that introduces a parameter $\lambda$ to he cost function of a ML model to punish it for choosing large scores. This allows `LASSO` to identify the best and worst features. These shortlisted features can then be used in further ML models.

*Note: A rolling window `LASSO` was used to ensure the features identified were applicable for the whole time frame rather than just a particular point in time. Also, the window chosen was made short in the hope that the data would approximately be linear in this short window.*

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

def rolling_window_lasso(window_size, step_size, alpha=0.01, target_variable='value_covidOccupiedMVBeds'):
    """
    Apply Lasso feature selection on a rolling window basis to identify important features
    based on their coefficients, considering a regularisation parameter.

    Parameters:
    window_size (int): The size of the rolling window in days.
    step_size (int): The step size for the rolling window.
    alpha (float): The regularisation strength; must be a positive float.
    target_variable (str): The name of the target variable to exclude from the Lasso model.

    Returns:
    feature_importance_df (pd.DataFrame): A DataFrame with average coefficients for the features across all windows.
    """

    data_df = combined_df.drop([target_variable], axis=1)  # Exclude the target variable
    target_df = combined_df[target_variable]  # Target variable data
    feature_importance_df = pd.DataFrame(0, index=data_df.columns, columns=['Coefficient'])

    scaler = StandardScaler()

    # Begin rolling window iteration
    for start in range(0, len(data_df) - window_size + 1, step_size):
        end = start + window_size
        window_data = data_df.iloc[start:end]
        window_target = target_df.iloc[start:end]

        window_data_scaled = scaler.fit_transform(window_data)
        
        # Applying Lasso
        lasso = LassoCV(alphas=[alpha], cv=5, random_state=42)
        lasso.fit(window_data_scaled, window_target)
        feature_importance_df['Coefficient'] += np.abs(lasso.coef_)


    return feature_importance_df




In [ ]:
#NOTE TO MARKER: THIS CELL WILL HAVE CONVERGENCE ISSUES SO BE CAREFUL WHEN RUNNING!!!!
#lasso_features_df = rolling_window_lasso(window_size=5, step_size=1, alpha=0.01, target_variable='value_covidOccupiedMVBeds')
#print(lasso_features_df.sort_values(by='Coefficient', ascending=False))

<div style="background-color:#C2F5DD">

## Principal Component Analysis (PCA) feature reduction

Since the `LASSO` feature did not work, another dimensionality reduction feature called PCA can be implemented. This converts the vector space into another space with a linear combination of the features. Thus, the features with the highest coefficients can be selected which captures most of the variance of the data. Again, a rolling PCA window is used, whilst summing the feature coefficients to ensure that the selected features are important over the whole time domain.

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns  # For plotting stuff

def rolling_window_pca(window_size, step_size, n_components=0.99, 
                       target_variable='value_covidOccupiedMVBeds',
                       plot=False, num_features_to_plot=5, display_scatter_matrix=False, random_features=False):
    """
    Apply PCA on a rolling window basis to calculate feature importance,
    excluding the target variable and focusing on the overall features for modelling.

    Parameters:
    window_size (int): The size of the rolling window in days.
    step_size (int): The step size for the rolling window.
    n_components (float): The amount of variance that PCA should retain.
    target_variable (str): The name of the target variable to exclude from the PCA.
    plot (bool): If True, plot the importances of the specified number of features.
    random_features (bool): If True, select features randomly for plotting instead of top features.
    display_scatter_matrix (bool): If True, display a scatter matrix of the top features.
    num_features_to_plot (int): Number of top features to display in the plot.

    Returns:
    feature_importance_df (DataFrame): A DataFrame with normalised importance scores for all features.
    """
    
    data_df = combined_df.drop([target_variable], axis=1)  # Exclude the target variable
    feature_importance_df = pd.DataFrame(0, index=data_df.columns, columns=['Importance'])
    scaler = StandardScaler()

    # Rolling window iteration
    for start in range(0, len(data_df) - window_size + 1, step_size):
        end = start + window_size
        window_data = data_df.iloc[start:end]

        data_scaled = scaler.fit_transform(window_data)
        pca = PCA(n_components=n_components, random_state=42)
        pca.fit(data_scaled)
        
        # Sum pca scores across time
        loadings_sum = np.sum(np.abs(pca.components_), axis=0)
        feature_importance_df['Importance'] += loadings_sum

    
    feature_importance_df['Importance'] /= feature_importance_df['Importance'].max()

    # Determine which features to plot
    if random_features:
        features_to_plot_df = feature_importance_df.sample(n=num_features_to_plot, random_state=42)
    else:
        features_to_plot_df = feature_importance_df.nlargest(num_features_to_plot, 'Importance')
    
    if plot:
        features_to_plot_df.sort_values(by='Importance', ascending=True).plot(kind='barh', figsize=(10, 8))
        plt.title(f'{len(features_to_plot_df)} Features Importance')
        plt.xlabel('Normalised Importance')
        plt.show()

    if display_scatter_matrix:
        top_features_names = features_to_plot_df.index
        scatter_matrix_df = data_df[top_features_names]
        pair_plot = sns.pairplot(scatter_matrix_df, height = 5)

        for ax in pair_plot.axes.flatten():
            ax.set_xlabel(ax.get_xlabel(), rotation=90)
            ax.set_ylabel(ax.get_ylabel(), rotation=0)
            ax.yaxis.labelpad = 300
            ax.xaxis.labelpad = 20
        #plt.tight_layout()
        plt.show()

    return feature_importance_df



In [ ]:
top_features_df = rolling_window_pca(window_size=7, step_size=1, n_components=0.99,
                                  target_variable='value_covidOccupiedMVBeds',
                                  plot=True, num_features_to_plot=5, display_scatter_matrix=True, random_features=True)

print('DEATH PLOTS BELOW:')

top_features_df_deaths = rolling_window_pca(window_size=7, step_size=1, n_components=0.99,
                                  target_variable='value_newDailyNsoDeathsByDeathDate',
                                  plot=True, num_features_to_plot=5, display_scatter_matrix=True, random_features=True)


<div style="background-color:#C2F5DD">

## Additional Feature Engineering

Once, the top features exlcuding the target feature of covid beds has been determined. Further feature engineering can be done. For example, just by inspection there are many variables above that are very similar/correlated but there is no reason to include all these features which represent the same thing. For example, the death tally over 28 days, 60 days or the death rate for those periods describe the same data but in different contexts. Having only one of them is enough to encapsulate the information whilst reducing the number of features at the same time. Inputting high dimensional data with highly correlated features would result in a ML model overfitting due to the computational complexity. We can check if any other features are highly correlated and remove them from the combined dataframe. This can be done to remove redudant features.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def remove_correlated_features(features_df, top_features_df, correlation_threshold=0.8, plot=False, num_features_to_plot=5,
                              random_features=False):
    """
    Identifies and removes highly correlated features based on the correlation threshold.
    Optionally plots a correlation heatmap of the most important features based on PCA.
    
    Parameters:
    features_df (DataFrame): DataFrame containing the features to check.
    top_features_df (DataFrame): DataFrame containing feature importances.
    correlation_threshold (float): Threshold above which a pair of features is considered highly correlated.
    plot (bool): If True, display a heatmap of the most correlated features.
    num_features_to_plot (int): Number of features to display in the plot.
    random_features (bool): If True, selects features randomly for plotting instead of top features.
    
    Returns:
    reduced_features_df (DataFrame): DataFrame with less correlated features.
    """
    #Calculating the correlation matrix
    features_df = features_df[top_features_df.index]
    corr_matrix = features_df.corr().abs()

    if plot:
        if random_features:
            features_to_plot = pd.Series(top_features_df.index).sample(n=num_features_to_plot, random_state=42)
        else:
            features_to_plot = top_features_df['Importance'].nlargest(num_features_to_plot).index

        sns.heatmap(corr_matrix.loc[features_to_plot, features_to_plot], cmap='coolwarm', cbar=True, linewidth= 0.5)
        plt.title('Correlation Matrix for Selected Features')
        plt.show()

    # Identify features that are highly correlated to drop
    to_drop = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if i == j:
                continue #expect correlation between similar features
            
            if corr_matrix.iloc[i, j] >= correlation_threshold:
                feature1 = corr_matrix.columns[i]
                feature2 = corr_matrix.columns[j]
                # Retain the more important feature
                if top_features_df.loc[feature1, 'Importance'] > top_features_df.loc[feature2, 'Importance']:
                    to_drop.append(feature1)
                else:
                    to_drop.append(feature2)

    reduced_features_df = features_df.drop(columns=to_drop)
    print(f"Selected {len(reduced_features_df.columns)} features: {reduced_features_df.columns}")
    return reduced_features_df


In [ ]:
reduced_df = remove_correlated_features(features_df=combined_df, top_features_df=top_features_df,
                                        correlation_threshold=0.8, plot=True, num_features_to_plot=5, random_features=True)

reduced_df_deaths = remove_correlated_features(features_df=combined_df, top_features_df=top_features_df_deaths,
                                        correlation_threshold=0.8, plot=False, num_features_to_plot=5, random_features=True)

<div style="background-color:#C2F5DD">

From the correlation matrix, it is obvious that there are numerous features that are correlated. The most uncorrelated features are the google movement features which makes sense because those are unique and not found in any other dataset. This provides a sanity check that the PCA and correlation matrices are working as intended. Now we order the dataframe by importance and select the data based on importance from the PCA. This will aid in narrowing down the most important uncorrelated features.

*Note to marker: In the correlation matrix heatmap, there may be whitespaces sometimes. This is not a cause for concern, all this implies is that the feature importance score for that particular feature from PCA was 0. Thus the correlation matrix calculated a bunch of NaNs leading to white spaces. This is not an issue because the features with low importance do not get selected for the ML regression model.*

In [ ]:
top_features_df.sort_values(by='Importance', ascending=False).head()

In [ ]:
def select_top_features(data_df, importance_df, num_features=15, target='covidOccupiedMVBeds'):
    """
    Selects the top columns from reduced_df based on the importance scores in top_features_df.

    Parameters:
    reduced_df (DataFrame): DataFrame with reduced features.
    top_features_df (DataFrame): DataFrame containing the importance scores of features.
    num_features (int): Number of top features to select.

    Returns:
    processed_df (DataFrame): DataFrame containing only the top features including the data.
    """
    
    sorted_features = importance_df['Importance'].sort_values(ascending=False).index
    available_features = [feature for feature in sorted_features if feature in data_df.columns] #check to see if it uncorrelated
    top_features = available_features[:num_features]
    processed_df = data_df[top_features]
    
    target = 'value_' + target
    temp_df = combined_df[target]
    processed_df = pd.concat([processed_df, temp_df], axis=1)

    return processed_df




In [ ]:
ML_df = select_top_features(data_df = reduced_df, importance_df=top_features_df, num_features=15)

ML_df_deaths = select_top_features(data_df = reduced_df_deaths, importance_df=top_features_df_deaths, num_features=15, target='newDailyNsoDeathsByDeathDate')

ML_df.head() #check to see if everything in order


<div style="background-color:#C2F5DD">

Now the data preprocessing has been completed. We can finally move on to calculate the time lag between the features and the target variable. Although it is worth noting the important of the days for the dataset. Maybe it will be worth doing the ML model with and without the additionally included day/month/year features.

<div style="background-color:#C2F5DD">

### Time lag between features
There will be a time lag between features changing and the actual effects being visible on the number of COVID beds required. To quantify this time lag the cross correlation between the COVID beds and the different features can be used to determine the time lag for which there will be the maximum cross correlation. The maximum time lag determined via research is roughly 11 days. This allows us to search a narrower range of times allowing for lower computational complexitiy. *Ref:* [Quantifying the Time-Lag Effects of Human
Mobility on the COVID-19 Transmission:
A Multi-City Study in China](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9262890). In this approach we will search for time lags of 100 days to be able to clearly observe the maximum shift although we are only expecting a shift of around 11 days.

When conducting the lag we only need to look forward in time i.e. start searching from 0 days onwards because we are expecting the features to have an effect on the covid beds rather than the covid beds having an effect on the feature. Thus, only positive time lags are considered. Furthermore, if the correlation was found to be more than 30 days then the lag was set to 0 because in those cases there was no clear trend of the features affecting the covid beds.

In addition, because the day/month/year should not have any correlation because the just represent time so the lag was set to 0 by default.


In [ ]:
def plot_cross_correlation(feature, df, target='value_covidOccupiedMVBeds', max_lag=100, display_plot = False):
    """
    Plot cross-correlation between a feature and a target variable over a range of lags.
    
    Parameters:
    df (DataFrame): DataFrame containing the time series data
    feature (str): Column name of the feature to correlate with the target
    target (str): Column name of the target variable
    max_lag (int): Maximum lag to compute the cross-correlation for

    Returns:
    optimal_lag (int): The optimal lag for that feature compared to the target
    """
    
    correlations = [] # List to store correlation coefficients

    if feature == target:
        return 0

    if ('day' in feature) or ('month' in feature) or ('year' in feature):
        return 0
    
    
    for lag in range(0, max_lag + 1): #search postive range
        shifted = df[feature].shift(lag)  # Shift the feature
        corr = shifted.corr(df[target])   # Calculate correlation with the target
        correlations.append(corr)

    if display_plot:
        plt.figure(figsize=(10, 5))
        plt.plot(range(0, max_lag + 1), correlations, marker='.', label = "Cross correlation")
        plt.axvline(x=0, color='k', linestyle='--', label='No Shift')
        plt.title(f'Cross-Correlation\nFeature: {feature} vs. {target}')
        plt.xlabel('Lag (Days)')
        plt.ylabel('Correlation Coefficient')
        plt.grid(True)
        plt.legend(loc = "best")
        plt.show()

    max_corr = max(correlations)  # Consider the maximum by absolute value
    optimal_lag = correlations.index(max_corr)
    print(f"Maximum correlation of {max_corr:.2f} occurs at lag {optimal_lag}.")

    if optimal_lag > 30: #the lag should not exceede one month
        return 0

    return optimal_lag


In [ ]:
shift_list = [plot_cross_correlation(feature, df = ML_df, display_plot=True) for feature in ML_df.columns]

shift_list_deaths = [plot_cross_correlation(feature, df = ML_df_deaths, display_plot=False, 
                                            target = 'value_newDailyNsoDeathsByDeathDate') for feature in ML_df_deaths.columns]

In [ ]:
print(shift_list) #check the list

In [ ]:
def shift_df(data_df, shifts):

    shifted_df = pd.DataFrame(index=data_df.index)
    for col, shift in zip(data_df.columns, shifts):
        shifted_df[col] = data_df[col].shift(shift, fill_value=0) #shifting and replacing gaps with 0s

    return shifted_df

In [ ]:
final_ML_df = shift_df(data_df=ML_df, shifts=shift_list) #check to see everything works

final_ML_df_deaths = shift_df(data_df=ML_df_deaths, shifts=shift_list_deaths)

final_ML_df.head()

<div style="background-color:#C2F5DD">

## ML Regression

Now that the data has been preprocessed and some irrelevant features removed, the data can start to be inputted into an appropriate model. For this task, because of the complexity of the features and the non-linear relationships within the data it is probably best to use `RandomForest` or `XGBoost` to capture these complex relations between the features and the target variable. Additionally, `RandomForest` and `XGBoost` can deal with multi-dimensional data and has the ability to select the best features for the regression task.

In [ ]:
final_ML_df_deaths.isna().sum() #check all is good

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


def rolling_window_regression(df, train_window, prediction_period, estimator, target_variable='value_covidOccupiedMVBeds', plot=False):
    """
    Apply regression models on a rolling window basis, predicting the target variable.

    Parameters:
    df (DataFrame): DataFrame containing the features and target variable.
    train_window (int): The size of the training window in days.
    prediction_period (int): The prediction period window length.
    estimator (sklearn estimator): The regression model to be used.
    target_variable (str): The name of the target variable.
    plot (bool): If True, plot the actual vs predicted values.
    
    Returns:
    rmse (float): The Root Mean Squared Error of the predictions.
    """
    predictions = []
    actuals = []
    prediction_indices = []
    
    # Prepare the data
    X = df.drop(target_variable, axis=1)
    y = df[target_variable]

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', estimator)
    ])


    # Rolling window iteration
    for i in tqdm(range(0, len(df) - train_window - prediction_period + 1, prediction_period), desc="Model Training"):

        end_train = i + train_window
        start_pred = end_train
        end_pred = start_pred + prediction_period

        
        if end_pred > len(df):
            end_pred = len(df)
        
        if end_train >= len(df):
            continue
        
        #print(f"end train: {end_train}, start_pred:{start_pred} and end pred:{end_pred}")
        
        
        train_X = X.iloc[i:end_train]
        train_y = y.iloc[i:end_train]
        test_X = X.iloc[start_pred:end_pred]
        test_y = y.iloc[start_pred:end_pred]

       
        pipeline.fit(train_X, train_y)

        
        y_pred = pipeline.predict(test_X) #comparing to the actual data
        predictions.extend(y_pred)
        actuals.extend(test_y.values)
        prediction_indices.extend(test_y.index)

    rmse = np.sqrt(mean_squared_error(actuals, predictions)) #best way to calculate rmse
    
    if plot:
        plt.figure(figsize=(12, 6))
        plt.plot(y.index, y, label='Actual')
        plt.plot(prediction_indices, predictions, label='Predictions')
        plt.axvline(x=y.index[train_window], color='r', linestyle='--', label='Start of predictions')
        plt.title('Actual vs Predicted Values')
        plt.xlabel('Date')
        plt.xticks(rotation=90)
        plt.ylabel(target_variable)
        plt.legend()
        plt.grid(True)
        plt.show()


    return rmse


In [ ]:
model_XG = XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=3)

rmse = rolling_window_regression(df=final_ML_df, train_window=14, prediction_period=7,
                                 estimator=model_XG, plot=True)

rmse_deaths = rolling_window_regression(df=final_ML_df_deaths, train_window=14, prediction_period=7,
                                 estimator=model_XG, plot=True, target_variable='value_newDailyNsoDeathsByDeathDate')
print(f'Root Mean Squared Error for Beds: {rmse:.2f} and for Deaths: {rmse_deaths:.2f}')


In [ ]:
#Compare with random forest to see which one is better
model_RF = RandomForestRegressor(n_estimators=100, max_depth = 3, random_state=42)

rmse = rolling_window_regression(df=final_ML_df, train_window=14, prediction_period=7,
                                 estimator=model_RF, plot=True)

rmse_deaths = rolling_window_regression(df=final_ML_df_deaths, train_window=14, prediction_period=7,
                                 estimator=model_RF, plot=True, target_variable='value_newDailyNsoDeathsByDeathDate')

print(f'Root Mean Squared Error for Beds: {rmse:.2f} and for Deaths: {rmse_deaths:.2f}')


<div style="background-color:#C2F5DD">

### Feature Engineering

From the scatter matrix plot previously done, when selecting most important features it was noticed that a lot of the features are highly skewed. Now to adjust this, a log transformation can be applied to make the feature data more uniform and gaussian-like distribution. This may improve the performace of the ML model. Below is the code to do this. The code has been written using `sklearn` packages such that the log transformer can be added to the `Pipeline` we created above.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class custom_log_transformer: #making custom class to put into Pipeline

    '''
        Custom transformation class that takes the log of the data. 

        Methods:
        fit: allow the log transformer to fit the data
        transform: allow the log transformer to change the data to log values
    '''

    def __init__(self, add_one=True):
        self.add_one = add_one
        

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        
        if self.add_one:
            return np.log(X)
        else:
            return np.log(X)



In [ ]:
#Let us see if this improves the model in any way



def rolling_window_regression_with_log(df, train_window, prediction_period, estimator, 
                                       target_variable='value_covidOccupiedMVBeds', plot=False):
    """
    Apply regression models on a rolling window basis, predicting the target variable.

    Parameters:
    df (DataFrame): DataFrame containing the features and target variable.
    train_window (int): The size of the training window in days.
    prediction_period (int): The prediction period window length.
    estimator (sklearn estimator): The regression model to be used.
    target_variable (str): The name of the target variable.
    plot (bool): If True, plot the actual vs predicted values.
    
    Returns:
    rmse (float): The Root Mean Squared Error of the predictions.
    """
    predictions = []
    actuals = []
    prediction_indices = []  # Store the indices of predictions

    # Prepare the data
    X = df.drop(target_variable, axis=1)
    y = df[target_variable]

    pipeline = Pipeline([ 
        ('scaler', StandardScaler()),
        ('log', custom_log_transformer()), #added in log transformer
        ('regressor', estimator)
    ])


    # Rolling window iteration
    for i in tqdm(range(0, len(df) - train_window - prediction_period + 1, prediction_period), desc="Model Training"):

        end_train = i + train_window
        start_pred = end_train
        end_pred = start_pred + prediction_period

        
        if end_pred > len(df):
            end_pred = len(df)
        
        if end_train >= len(df):
            continue
        
        #print(f"end train: {end_train}, start_pred:{start_pred} and end pred:{end_pred}")
        
        
        train_X = X.iloc[i:end_train]
        train_y = y.iloc[i:end_train]
        test_X = X.iloc[start_pred:end_pred]
        test_y = y.iloc[start_pred:end_pred]

       
        pipeline.fit(train_X, train_y)

        
        y_pred = pipeline.predict(test_X) #comparing to the actual data
        predictions.extend(y_pred)
        actuals.extend(test_y.values)
        prediction_indices.extend(test_y.index)

    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    
    if plot:
        plt.figure(figsize=(12, 6))
        plt.plot(y.index, y, label='Actual')
        #prediction_start_index = y.index[train_window + prediction_period - 2:] 
        #prediction_start_index = y.index[len(predictions)]
        plt.plot(prediction_indices, predictions, label='Predictions')
        plt.axvline(x=y.index[train_window + prediction_period - 1], color='r', linestyle='--', label='Start of predictions')
        plt.title('Actual vs Predicted Values')
        plt.xlabel('Date')
        plt.xticks(rotation=90)
        plt.ylabel(target_variable)
        plt.legend()
        plt.grid(True)
        plt.show()


    return rmse


In [ ]:
#Log transform did not work!

#model_RF = RandomForestRegressor(n_estimators=1000, max_depth = 3, random_state=42)

#rmse = rolling_window_regression_with_log(df=final_ML_df, train_window=14, prediction_period=7,
                                 #estimator=model_RF, plot=True)

#rmse_deaths = rolling_window_regression_wth_log(df=final_ML_df_deaths, train_window=14, prediction_period=7,
                                 #estimator=model_RF, plot=True, target_variable='value_newDailyNsoDeathsByDeathDate')

<div style="background-color:#C2F5DD">

*Note: Tried to implement the log transformer, but there were a lot of values close to 0 that were becoming inf or too large to store in float64 arrays. Thus, this approach was abandoned.*

<div style="background-color:#C2F5DD">

### Hyperparameter tuning

From the above examples, it can be seen that the `Random Forest Regressor` performs slighly better than the `XGBoost Regressor`. Thus, we can safely proceede with the `Random Forest Regressor` from here onwards. The `Random Forest Regressor` can be fine tuned in a couple of ways: firstly, hyperparameter tuning can be done to ensure that the model is fine tuned for this particular dataset. Secondly, the training and testing period above of 14 days and 7 days respectively was chosen arbitrarily so this can be changed to see the optimal training and testing periods for the rolling window. Finally, it is worth removing the days/months/years column or adding in more features to see if the model performance increases.

<div style="background-color:#C2F5DD">

Now the training window and the prediction period needs to be optimised such that the training window is not too far back in the future such that the model learns trends from previous times and the prediction period short enough such that the trends do not deviate too much from the model predictions. For these parts it would be ideal to use the HPC cluster to fine tune through all the combinations but in the interest of time we can perform a rough search outlined below. Also in the interest of time, the window was optimised just for one model instead of both. This will half the number of computations required.

*Note: Feel free to try and optimise the windows for both models, the code is in there.*

In [ ]:
#Optimising the training window and prediction period
train_window_list = [i for i in range(14, 43, 7)]
prediction_period_list = [i for i in range(1, 8, 2)]

best_rmse = 1e9 #some large values
#best_rmse_deaths = 1e9 

ideal_train_window = 0
ideal_prediction_period = 0

#ideal_train_window_deaths = 0
#ideal_prediction_period_deaths = 0

model_RF = RandomForestRegressor(n_estimators=100, random_state=42) #simpler model to speed up process

for train_window in train_window_list:
    for prediction_period in prediction_period_list:
        rmse = rolling_window_regression(df=final_ML_df, train_window=train_window, prediction_period=prediction_period,
                                 estimator=model_RF, plot=False)

        if rmse < best_rmse:
            best_rmse = rmse
            ideal_train_window = train_window
            ideal_prediction_period = prediction_period

        #rmse_deaths = rolling_window_regression(df=final_ML_df_deaths, train_window=train_window, prediction_period=prediction_period,
                                 #estimator=model_RF, plot=False, target_variable='value_newDailyNsoDeathsByDeathDate')

        # if rmse_deaths < best_rmse_deaths:
        #     best_rmse_deaths = rmse_deaths
        #     ideal_train_window = train_window
        #     ideal_prediction_period = prediction_period

print(f'Best rmse is {best_rmse:.2f} for training window: {ideal_train_window} and prediction period: {ideal_prediction_period}')
#print(f'COVID Deaths - best rmse is {best_rmse_deaths} for training window: {ideal_train_widnow_deaths} and prediction period: {ideal_prediction_period_deaths}')

<div style="background-color:#C2F5DD">

Now that the optimal training window and prediction window has been determined, we can proceede to fine-tune the hyperparameters for the model. Again, in the interest of time we can loop over some of the parameters, coarsely rather than do some fine search for the optimal hyperparameters. Ideally, we would have used `GridSearchCV` to search the parameter space but in our case a simple nested for loop will suffice. So, for `Random Forest Regressor` there are many hyperparameters that can be tuned, but the most important ones are the n_estimators and the max_depth. Again, tuning for the beds and then applying to the covid deaths. Not ideal, but concept is same. This approach is only used to save computational time.

In [ ]:
n_estimators_list = [100, 200, 300]
max_depth_list = [3,4,5] #small grid for less computational cost. Not ideal.

# n_estimators_list = [i for i in range(100, 2000, 250)]
# max_depth_list = [i for i in range(1,10)] #better search, can use this if got high computation power.

final_rmse = 1e9
ideal_n_estimators = 0
ideal_max_depth = 0

# final_rmse_deaths = 1e9
# ideal_n_estimators_deaths = 0
# ideal_max_depth_deaths = 0


for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        new_model_RF = RandomForestRegressor(n_estimators=n_estimators, max_depth = max_depth, random_state=42, n_jobs=-1)
        rmse = rolling_window_regression(df=final_ML_df, train_window=ideal_train_window, prediction_period=ideal_prediction_period,
                                 estimator=new_model_RF, plot=False)

        if rmse < final_rmse:
            final_rmse = rmse
            ideal_n_estimators = n_estimators
            ideal_max_depth = max_depth

        
        #rmse_deaths = rolling_window_regression(df=final_ML_df, train_window=ideal_train_window, prediction_period=ideal_prediction_period,
                                 #estimator=model_RF, plot=False, target_variable='value_newDailyNsoDeathsByDeathDate')
        

        # if rmse_deaths < final_rmse_deaths:
        #     final_rmse_deaths = rmse_deaths
        #     ideal_n_estimators_deaths = n_estimators
        #     ideal_max_depth_deaths = max_depth

print(f'The best rmse is {final_rmse:.2f} for best max depth: {ideal_max_depth} and the best n estimators:{ideal_n_estimators}')

In [ ]:
#Now it is time to do the final plot with our model

model_RF = RandomForestRegressor(n_estimators=ideal_n_estimators, max_depth = ideal_max_depth, random_state=42)

rmse = rolling_window_regression(df=final_ML_df, train_window=ideal_train_window, prediction_period=ideal_prediction_period,
                                 estimator=model_RF, plot=True)

rmse_deaths = rolling_window_regression(df=final_ML_df_deaths, train_window=ideal_train_window, prediction_period=ideal_prediction_period,
                                 estimator=model_RF, plot=True, target_variable='value_newDailyNsoDeathsByDeathDate')

print(f'Root Mean Squared Error for Beds: {rmse:.2f} and for Deaths: {rmse_deaths:.2f}')

<div style="background-color:#C2F5DD">

## Conclusion

All in all, the model predicted the number of COVID beds and deaths required fairly well given the data. Ideally, with more computing power, the training and testing window and the hyperparameters could have been found more accurately. To improve the code, all the data preprocessing and the training could have been combined into one ML pipeline that can be run on all the data. Furthermore, we could have compared the PCA reduction with a basic Random Forest feature importance selection to cross-check whether PCA was picking up the most important results or not.

Given more time, some of the features could have been combined or multiplied together to see how the results are affected. In addition, more feature could have incorporated by examining the optimal boundary to set the cross-correlation threshold when deciding if a dataset is correlated or not. This could have been done by varying the cross-correlation threshold and observing how the ML model improves over time.